In [15]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

In [ ]:
#### CALL BERDASARKAN LAG ####

In [7]:
# Load training and testing data
train_data = pd.read_excel("/Users/shannenwibisono/Desktop/-SKRIPSI-/S&P500/SP500 Training Testing.xlsx", sheet_name="Training Data")
test_data = pd.read_excel("/Users/shannenwibisono/Desktop/-SKRIPSI-/S&P500/SP500 Training Testing.xlsx", sheet_name="Testing Data")

In [8]:
# Define the LightGBM parameters
params = {
    'boost_from_average': True,
    'colsample_bytree': 0.5,
    'fair_c': 1,
    'huber_delta': 1,
    'learning_rate': 0.05,
    'max_bin': 255,
    'max_delta_step': 0.7,
    'max_depth': -1,  # None is represented by -1 in LightGBM
    'min_child_samples': 10,
    'min_child_weight': 5,
    'min_split_gain': 0,
    'n_estimators': 2500,
    'num_leaves': 64,
    'objective': 'gamma',
    'reg_alpha': 0,
    'reg_lambda': 0,
    'subsample': 1,
    'subsample_for_bin': 50000,
    'subsample_freq': 1,
    'tweedie_variance_power': 1.5,
    'early_stopping_round': 200  # Set early stopping in parameters
}

# Define lag configurations
lags = {
    'lag1': ['RVlag1', 'BVlag1'],
    'lag7': ['RVlag7', 'BVlag7', 'SJVlag7'],
    'lag15': ['RVlag15', 'BVlag15', 'SJVlag15'],
    'lag30': ['RVlag30', 'BVlag30', 'SJVlag30']
}

In [9]:
# Output file path
output_file = "/Users/shannenwibisono/Desktop/-SKRIPSI-/Results LGBM/Results_LightGBM_SPCall_Lags.xlsx"

# Create a workbook to save results
wb = Workbook()
ws_summary = wb.active
ws_summary.title = "Summary"
ws_summary.append(["Lag", "RMSE", "MAPE"])  # Add headers for summary

In [10]:
# Loop through each lag configuration
for lag_name, features in lags.items():
    print(f"\nTesting with lag: {lag_name}")
    
    # Select relevant features
    selected_features = ['Price.', 'Maturity', 'Strike'] + features
    
    # Prepare features and target
    x_train = train_data[selected_features].values
    y_train = train_data['Last'].values
    x_test = test_data[selected_features].values
    y_test = test_data['Last'].values
    
    # Create LightGBM datasets
    train_set = lgb.Dataset(x_train, label=y_train)
    test_set = lgb.Dataset(x_test, label=y_test, reference=train_set)
    
    # Train the model
    model = lgb.train(
        params, 
        train_set, 
        valid_sets=[train_set, test_set], 
        valid_names=["train", "test"], 
        num_boost_round=2500
    )
    
    # Make predictions
    y_pred = model.predict(x_test, num_iteration=model.best_iteration)
    
    # Calculate RMSE and MAPE
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    print(f"Lag: {lag_name} | RMSE: {rmse:.4f} | MAPE: {mape:.2f}%")
    
    # Save results in a new sheet
    results = pd.DataFrame({
        'Date': test_data['Time'],  # Adjust 'Time' to your date column name
        'Actual': y_test,
        'Predicted': y_pred,
        'RMSE': [rmse] * len(y_test),  # Add RMSE for consistency
        'MAPE': [mape] * len(y_test)  # Add MAPE for consistency
    })
    
    # Add summary results
    ws_summary.append([lag_name, rmse, mape])
    
    # Add sheet for this lag
    ws = wb.create_sheet(title=lag_name)
    for r in dataframe_to_rows(results, index=False, header=True):
        ws.append(r)

# Save the workbook
wb.save(output_file)
print(f"All results saved to Excel file: {output_file}")


Testing with lag: lag1
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000690 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1225
[LightGBM] [Info] Number of data points in the train set: 28473, number of used features: 5
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 6.276901
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2478]	train's gamma: 5.66022	test's gamma: 5.69819
Lag: lag1 | RMSE: 40.5196 | MAPE: 8.76%

Testing with lag: lag7
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000332 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1466
[LightGBM] [Info] Number of data points in the train set: 28473, number of used features: 6
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 6.276901
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2496]	train's gamma: 5.66091	test's gamma: 5.69831
Lag: lag7 | RMSE: 45.7753 | MAPE: 9.14%

Testing with lag: lag15
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000106 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1461
[LightGBM] [Info] Number of data points in the train set: 28473, number of used features: 6
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 6.276901
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2500]	train's gamma: 5.66077	test's gamma: 5.69815
Lag: lag15 | RMSE: 46.3653 | MAPE: 8.96%

Testing with lag: lag30
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000115 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1457
[LightGBM] [Info] Number of data points in the train set: 28473, number of used features: 6
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 6.276901
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2497]	train's gamma: 5.6607	test's gamma: 5.69771
Lag: lag30 | RMSE: 42.3481 | MAPE: 8.64%
All results saved to Excel file: /Users/shannenwibisono/Desktop/-SKRIPSI-/Results LGBM/Results_LightGBM_SPCall_Lags.xlsx


In [ ]:
#### CALL BERDASARKAN JENIS VARIATION ####

In [37]:
# Load training and testing data
train_data = pd.read_excel("/Users/shannenwibisono/Desktop/-SKRIPSI-/S&P500/SP500 Training Testing.xlsx", sheet_name="Training Data")
test_data = pd.read_excel("/Users/shannenwibisono/Desktop/-SKRIPSI-/S&P500/SP500 Training Testing.xlsx", sheet_name="Testing Data")

# Define the LightGBM parameters
params = {
    'boost_from_average': True,
    'colsample_bytree': 0.5,
    'fair_c': 1,
    'huber_delta': 1,
    'learning_rate': 0.05,
    'max_bin': 255,
    'max_delta_step': 0.7,
    'max_depth': -1,  # None is represented by -1 in LightGBM
    'min_child_samples': 10,
    'min_child_weight': 5,
    'min_split_gain': 0,
    'n_estimators': 2500,
    'num_leaves': 64,
    'objective': 'gamma',
    'reg_alpha': 0,
    'reg_lambda': 0,
    'subsample': 1,
    'subsample_for_bin': 50000,
    'subsample_freq': 1,
    'tweedie_variance_power': 1.5,
    'early_stopping_round': 200  # Set early stopping in parameters
}


In [38]:
# Define the variations and their respective lag features
variations = {
    'SJV': ['SJVlag1', 'SJVlag7', 'SJVlag15', 'SJVlag30'],
    'BV': ['BVlag1', 'BVlag7', 'BVlag15', 'BVlag30'],
    'RV': ['RVlag1', 'RVlag7', 'RVlag15', 'RVlag30']
}

In [39]:
# Output file path
output_file = "/Users/shannenwibisono/Desktop/-SKRIPSI-/Results LGBM/Results_LightGBM_SPCall_Variations.xlsx"

In [40]:
# Create a workbook to save results
wb = Workbook()
ws_summary = wb.active
ws_summary.title = "Summary"
ws_summary.append(["Variation", "RMSE", "MAPE"])  # Add headers for summary

In [41]:
# Loop through each variation
for var_name, features in variations.items():
    print(f"\nTesting with variation: {var_name}")
    
    # Select relevant features
    selected_features = ['Price.', 'Maturity', 'Strike'] + features
    
    # Prepare features and target
    x_train = train_data[selected_features].values
    y_train = train_data['Last'].values
    x_test = test_data[selected_features].values
    y_test = test_data['Last'].values
    
    # Create LightGBM datasets
    train_set = lgb.Dataset(x_train, label=y_train)
    test_set = lgb.Dataset(x_test, label=y_test, reference=train_set)
    
    # Train the model
    model = lgb.train(
        params, 
        train_set, 
        valid_sets=[train_set, test_set], 
        valid_names=["train", "test"], 
        num_boost_round=2500
    )
    
    # Make predictions
    y_pred = model.predict(x_test, num_iteration=model.best_iteration)
    
    # Calculate RMSE and MAPE
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    print(f"Variation: {var_name} | RMSE: {rmse:.4f} | MAPE: {mape:.2f}%")
    
    # Save results in a new sheet
    results = pd.DataFrame({
        'Date': test_data['Time'],  # Adjust 'Time' to your date column name
        'Actual': y_test,
        'Predicted': y_pred,
        'RMSE': [rmse] * len(y_test),  # Add RMSE for consistency
        'MAPE': [mape] * len(y_test)  # Add MAPE for consistency
    })
    
    # Add summary results
    ws_summary.append([var_name, rmse, mape])
    
    # Add sheet for this variation
    ws = wb.create_sheet(title=var_name)
    for r in dataframe_to_rows(results, index=False, header=True):
        ws.append(r)

# Save the workbook
wb.save(output_file)
print(f"All results saved to Excel file: {output_file}")


Testing with variation: SJV
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000759 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1711
[LightGBM] [Info] Number of data points in the train set: 28473, number of used features: 7
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 6.276901
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2497]	train's gamma: 5.65999	test's gamma: 5.69773
Variation: SJV | RMSE: 40.2223 | MAPE: 8.50%

Testing with variation: BV
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000148 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1700
[LightGBM] [Info] Number of data points in the train set: 28473, number of used features: 7
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 6.276901
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2500]	train's gamma: 5.65998	test's gamma: 5.69745
Variation: BV | RMSE: 40.1351 | MAPE: 8.34%

Testing with variation: RV
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000751 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1705
[LightGBM] [Info] Number of data points in the train set: 28473, number of used features: 7
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 6.276901
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2496]	train's gamma: 5.66	test's gamma: 5.69791
Variation: RV | RMSE: 40.2055 | MAPE: 8.57%
All results saved to Excel file: /Users/shannenwibisono/Desktop/-SKRIPSI-/Results LGBM/Results_LightGBM_SPCall_Variations.xlsx


In [ ]:
#### PUT BERDASARKAN LAG ####

In [42]:
# Read training and testing data for Put options
train_data_put = pd.read_excel("/Users/shannenwibisono/Desktop/-SKRIPSI-/S&P500/SP500 Training Testing PUT.xlsx", sheet_name="Training Data Put")
test_data_put = pd.read_excel("/Users/shannenwibisono/Desktop/-SKRIPSI-/S&P500/SP500 Training Testing PUT.xlsx", sheet_name="Testing Data Put")

In [43]:
# Define the LightGBM parameters
params = {
    'boost_from_average': True,
    'colsample_bytree': 0.5,
    'fair_c': 1,
    'huber_delta': 1,
    'learning_rate': 0.05,
    'max_bin': 255,
    'max_delta_step': 0.7,
    'max_depth': -1,  # None is represented by -1 in LightGBM
    'min_child_samples': 10,
    'min_child_weight': 5,
    'min_split_gain': 0,
    'n_estimators': 2500,
    'num_leaves': 16,
    'objective': 'gamma',
    'reg_alpha': 0,
    'reg_lambda': 0,
    'subsample': 1,
    'subsample_for_bin': 50000,
    'subsample_freq': 1,
    'tweedie_variance_power': 1.5,
    'early_stopping_round': 200  # Set early stopping in parameters
}

# Define lag configurations
lags = {
    'lag1': ['RVlag1', 'BVlag1'],
    'lag7': ['RVlag7', 'BVlag7', 'SJVlag7'],
    'lag15': ['RVlag15', 'BVlag15', 'SJVlag15'],
    'lag30': ['RVlag30', 'BVlag30', 'SJVlag30']
}

In [44]:
# Output file path
output_file = "/Users/shannenwibisono/Desktop/-SKRIPSI-/Results LGBM/Results_LightGBM_SPPut_Lags.xlsx"

# Create a workbook to save results
wb = Workbook()
ws_summary = wb.active
ws_summary.title = "Summary"
ws_summary.append(["Lag", "RMSE", "MAPE"])  # Add headers for summary

In [45]:
# Loop through each lag configuration
for lag_name, features in lags.items():
    print(f"\nTesting with lag: {lag_name}")
    
    # Select relevant features
    selected_features = ['Price.', 'Maturity', 'Strike'] + features
    
    # Prepare features and target
    x_train = train_data_put[selected_features].values
    y_train = train_data_put['Last'].values
    x_test = test_data_put[selected_features].values
    y_test = test_data_put['Last'].values
    
    # Create LightGBM datasets
    train_set = lgb.Dataset(x_train, label=y_train)
    test_set = lgb.Dataset(x_test, label=y_test, reference=train_set)
    
    # Train the model
    model = lgb.train(
        params, 
        train_set, 
        valid_sets=[train_set, test_set], 
        valid_names=["train", "test"], 
        num_boost_round=2500
    )
    
    # Make predictions
    y_pred = model.predict(x_test, num_iteration=model.best_iteration)
    
    # Calculate RMSE and MAPE
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    print(f"Lag: {lag_name} | RMSE: {rmse:.4f} | MAPE: {mape:.2f}%")
    
    # Save results in a new sheet
    results = pd.DataFrame({
        'Date': test_data_put['Time'],  # Adjust 'Time' to your date column name
        'Actual': y_test,
        'Predicted': y_pred,
        'RMSE': [rmse] * len(y_test),  # Add RMSE for consistency
        'MAPE': [mape] * len(y_test)  # Add MAPE for consistency
    })
    
    # Add summary results
    ws_summary.append([lag_name, rmse, mape])
    
    # Add sheet for this lag
    ws = wb.create_sheet(title=lag_name)
    for r in dataframe_to_rows(results, index=False, header=True):
        ws.append(r)

# Save the workbook
wb.save(output_file)
print(f"All results saved to Excel file: {output_file}")


Testing with lag: lag1
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001109 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1233
[LightGBM] [Info] Number of data points in the train set: 38394, number of used features: 5
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 4.684843
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2466]	train's gamma: 3.98452	test's gamma: 3.97518
Lag: lag1 | RMSE: 25.4865 | MAPE: 11.96%

Testing with lag: lag7
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000141 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1480
[LightGBM] [Info] Number of data points in the train set: 38394, number of used features: 6
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 4.684843
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2500]	train's gamma: 3.98553	test's gamma: 3.97581
Lag: lag7 | RMSE: 27.3632 | MAPE: 12.44%

Testing with lag: lag15
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000891 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1476
[LightGBM] [Info] Number of data points in the train set: 38394, number of used features: 6
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 4.684843
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2489]	train's gamma: 3.98527	test's gamma: 3.97536
Lag: lag15 | RMSE: 25.6012 | MAPE: 12.23%

Testing with lag: lag30
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000137 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1465
[LightGBM] [Info] Number of data points in the train set: 38394, number of used features: 6
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 4.684843
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2500]	train's gamma: 3.985	test's gamma: 3.97545
Lag: lag30 | RMSE: 26.3477 | MAPE: 12.18%
All results saved to Excel file: /Users/shannenwibisono/Desktop/-SKRIPSI-/Results LGBM/Results_LightGBM_SPPut_Lags.xlsx


In [ ]:
#### PUT BERDASARKAN JENIS VARIATION ####

In [46]:
# Read training and testing data for Put options
train_data_put = pd.read_excel("/Users/shannenwibisono/Desktop/-SKRIPSI-/S&P500/SP500 Training Testing PUT.xlsx", sheet_name="Training Data Put")
test_data_put = pd.read_excel("/Users/shannenwibisono/Desktop/-SKRIPSI-/S&P500/SP500 Training Testing PUT.xlsx", sheet_name="Testing Data Put")

# Define the LightGBM parameters
params = {
    'boost_from_average': True,
    'colsample_bytree': 0.5,
    'fair_c': 1,
    'huber_delta': 1,
    'learning_rate': 0.05,
    'max_bin': 255,
    'max_delta_step': 0.7,
    'max_depth': -1,  # None is represented by -1 in LightGBM
    'min_child_samples': 10,
    'min_child_weight': 5,
    'min_split_gain': 0,
    'n_estimators': 2500,
    'num_leaves': 16,
    'objective': 'gamma',
    'reg_alpha': 0,
    'reg_lambda': 0,
    'subsample': 1,
    'subsample_for_bin': 50000,
    'subsample_freq': 1,
    'tweedie_variance_power': 1.5,
    'early_stopping_round': 200  # Set early stopping in parameters
}

In [47]:
# Define the variations and their respective lag features
variations = {
    'SJV': ['SJVlag1', 'SJVlag7', 'SJVlag15', 'SJVlag30'],
    'BV': ['BVlag1', 'BVlag7', 'BVlag15', 'BVlag30'],
    'RV': ['RVlag1', 'RVlag7', 'RVlag15', 'RVlag30']
}

In [48]:
# Output file path
output_file = "/Users/shannenwibisono/Desktop/-SKRIPSI-/Results LGBM/Results_LightGBM_SPPut_Variations.xlsx"

In [49]:
# Create a workbook to save results
wb = Workbook()
ws_summary = wb.active
ws_summary.title = "Summary"
ws_summary.append(["Variation", "RMSE", "MAPE"])  # Add headers for summary

In [50]:
# Loop through each variation
for var_name, features in variations.items():
    print(f"\nTesting with variation: {var_name}")
    
    # Select relevant features
    selected_features = ['Price.', 'Maturity', 'Strike'] + features
    
    # Prepare features and target
    x_train = train_data_put[selected_features].values
    y_train = train_data_put['Last'].values
    x_test = test_data_put[selected_features].values
    y_test = test_data_put['Last'].values
    
    # Create LightGBM datasets
    train_set = lgb.Dataset(x_train, label=y_train)
    test_set = lgb.Dataset(x_test, label=y_test, reference=train_set)
    
    # Train the model
    model = lgb.train(
        params, 
        train_set, 
        valid_sets=[train_set, test_set], 
        valid_names=["train", "test"], 
        num_boost_round=2500
    )
    
    # Make predictions
    y_pred = model.predict(x_test, num_iteration=model.best_iteration)
    
    # Calculate RMSE and MAPE
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    print(f"Variation: {var_name} | RMSE: {rmse:.4f} | MAPE: {mape:.2f}%")
    
    # Save results in a new sheet
    results = pd.DataFrame({
        'Date': test_data_put['Time'],  # Adjust 'Time' to your date column name
        'Actual': y_test,
        'Predicted': y_pred,
        'RMSE': [rmse] * len(y_test),  # Add RMSE for consistency
        'MAPE': [mape] * len(y_test)  # Add MAPE for consistency
    })
    
    # Add summary results
    ws_summary.append([var_name, rmse, mape])
    
    # Add sheet for this variation
    ws = wb.create_sheet(title=var_name)
    for r in dataframe_to_rows(results, index=False, header=True):
        ws.append(r)

# Save the workbook
wb.save(output_file)
print(f"All results saved to Excel file: {output_file}")


Testing with variation: SJV
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000549 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1718
[LightGBM] [Info] Number of data points in the train set: 38394, number of used features: 7
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 4.684843
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2500]	train's gamma: 3.9841	test's gamma: 3.97477
Variation: SJV | RMSE: 23.0338 | MAPE: 11.72%

Testing with variation: BV
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000683 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1720
[LightGBM] [Info] Number of data points in the train set: 38394, number of used features: 7
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 4.684843
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2499]	train's gamma: 3.98416	test's gamma: 3.97494
Variation: BV | RMSE: 24.2304 | MAPE: 11.67%

Testing with variation: RV
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000165 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1720
[LightGBM] [Info] Number of data points in the train set: 38394, number of used features: 7
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 4.684843
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2499]	train's gamma: 3.98416	test's gamma: 3.97495
Variation: RV | RMSE: 22.8126 | MAPE: 11.73%
All results saved to Excel file: /Users/shannenwibisono/Desktop/-SKRIPSI-/Results LGBM/Results_LightGBM_SPPut_Variations.xlsx
